# Домашнее задание 2
Гончаров Алексей, 274 группа, ФУПМ, 5 курс

In [1]:
import pandas as pd
import numpy as np

//anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


Загрузим данные и посмотрим на сообщения

In [2]:
data = pd.read_csv('Emails.csv')
text = data.RawText
text[0]

'UNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739545\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.\nRELEASE IN FULL\nFrom: Sullivan, Jacob J <Sullivan11@state.gov>\nSent: Wednesday, September 12, 2012 10:16 AM\nTo:\nSubject: FW: Wow\nFrom: Brose, Christian (Armed Services) (mailto:Christian_Brose@armed-servic,essenate.govi\nSent: Wednesday, September 12, 2012 10:09 AM\nTo: Sullivan, Jacob J\nSubject: Wow\nWhat a wonderful, strong and moving statement by your boss. please tell her how much Sen. McCain appreciated it. Me\ntoo\nUNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739545\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045247\n\x0c'

Далее приводится итоговый получившийся код. Для более простой проверки удалил все неудачные или сравнимые по качеству попытки.

Текст обрабатывался следующим образом:
    0. Письмо резалось по слову UNCLASSIFIED. Таким образом выкидывались все письма, стоящие в копии, или же которые пересылались как то, на что получен ответ.
    1. Удалялись все не-буквы
    2. Удалялись все слова, задающие шапку письма (To, From, Attachment и прочее)
    3. Удалялись стандартные стоп-слова
    4. Слова лемматизировались
    5. Выделялись 2-Коллокации
Стэмминг и другие типы лемматизации не повышали качества. Выделение n-gramm не повышало качество по сравнению со случаем применения 2-коллокаций. 3-Коллокации также не дали существенный прирост качества. Использование цифр ухудшало качество резко. Использование полных текстов также понижало качество существенно.

Далее применялся стандартный CountVectorizer. Применение TfIdf существенно не меняло качество полученных кластеров.
После этого выделялись стоп-слова, встречающиеся более чем в 10% писем. Они тоже удалялись.
Проводилась классификация стандартным методом K-means. Выбранное число кластеров 25. При бОльшем количестве кластеров много кластеров с 1 текстом. При меньшем количестве теряется качество интерпретации. 
Другие методы кластеризации также применялись, но качество никак не менялось, иногда становилось хуже. 

В конце работы приводятся предположения по совершенствованию подхода, которые я не успел реализовать, так как компилляция кода происходит довольно долго.

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.collocations import *


bigram_measures = nltk.collocations.BigramAssocMeasures()
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

Все числа незначимы для последующей интерпретации. Оставим только символы слов. При этом удалим все слова, задающие шапку письма

In [4]:
def cut(message):
    row = message.split('UNCLASSIFIED')[1]#.split('UNCLASSIFIED')[0]
    return re.sub(r'[^\w\s]+|[\d]+', r'',row).strip().replace('To','').replace('B','').replace('Re','').replace('Cc','').replace('Subject','').replace('Attachments','').replace('\n',' ').replace('Original Message','').replace('  ',' ').lower()

К словам из сообщения добавим 10 лучшим коллокаций из 2х слов. Также выкинем все стоп-слова из текста, соединив их нижним подчеркиванием. Применим лемматизацию к одиночным словам

In [5]:
def feature_extr(message):
    row = cut(message)
    words = row.split(' ')
    row = np.array(words)[np.array(words)!='']
    row = np.array([x for x in row if x not in stopwords.words("english")])
    finder = BigramCollocationFinder.from_words(row)
    row = lemmatizer.lemmatize(' '.join((row)))
    return ' '.join((row,' '.join((['_'.join((x)) for x in finder.nbest(bigram_measures.pmi, 10)]))))

Применим это преобразование ко всем текстам, кроме одного пустого

In [6]:
new_text = []
for i in range(len(text)):
    if i!=1930:
        new_text.append(feature_extr(text[i]))
    else:
        new_text.append('zero')

Теперь будем обрабатывать полученный текст как мешок слов. Так как коллакации мы уже выделили, то биграммы не нужно учитывать. Применим CountVectorizer со стандартными настройками

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

cnt = CountVectorizer()

new_text2 = cnt.fit_transform(new_text)
dictionary = cnt.vocabulary_
vocabulary = np.array(cnt.get_feature_names())

Выведем слова, встречающиеся более чем в 10% датасета.

In [8]:
per = np.array(new_text2.todense() > 0).sum(axis=0)/float(new_text2.shape[0])
stop_words = vocabulary[per>0.1]
print stop_words

[u'abedin' u'abedin_huma' u'abedinhstategov' u'also' u'august' u'back'
 u'c_date' u'call' u'case' u'case_f' u'cheryl' u'could' u'date'
 u'date_release' u'december' u'department' u'department_state' u'doc'
 u'doc_c' u'f_doc' u'friday' u'full' u'fw' u'fyi' u'get' u'good'
 u'hdrclintonemailcom' u'hrodclintonemailcom' u'huma' u'im' u'jacob'
 u'know' u'last' u'like' u'may' u'meeting' u'mills' u'millscdstategov'
 u'monday' u'need' u'new' u'one' u'part' u'pm' u'president' u'release'
 u'said' u'saturday' u'secretary' u'see' u'sent' u'september' u'state'
 u'sullivan' u'sullivanjjstategov' u'sunday' u'thursday' u'time' u'today'
 u'tuesday' u'us' u'want' u'wednesday' u'well' u'work' u'would']


Проведем кластеризацию полученной матрицы частот слов и коллокаций. Опытным путем подобрали количество кластеров: 25 (подбирал на уменьшенном датасете). Не привожу все выкладки, так как на полном датасете работает слишком долго. Для бОльшего количества кластеров много кластеров, где присутствует 1-2 элемента

In [33]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=25)
clf.fit(new_text2)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=25, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

Визуализировать датасет решил путем вывода всех наиболее часто встречающихся слов в предложении - центре кластера. Выводим слова из словаря согласно количеству их встречаемости в самом центре кластера.
При этом не выводим те слова, которые уже встретились в других кластерах до этого. Выводим по 20 слов. 

In [34]:
center_unic_voc = [0]*25
all_used_text = []
for j in range(0,25):
    center_unic_voc[j]=[]
    i=0
    number = 0
    while i<=20:
            k = clf.cluster_centers_[j].argsort()[-number]
            number = number + 1
            if vocabulary[k] not in all_used_text and vocabulary[k] not in stop_words:
                all_used_text.append(vocabulary[k])
                center_unic_voc[j].append(vocabulary[k])
                i = i + 1

Для каждого кластера выводятся топ 5 слов, которые встречаются там чаще всего и не входят в топ слов. После этих 5 слов выводятся самые частые 20 слов. Они раскрывают тему. 

In [36]:
for i in np.bincount(clf.labels_).argsort()[::-1]:
    
    k=center_unic_voc[i]
    print 'cluster:',np.bincount(clf.labels_)[i],', '.join((k[:5]))
    print ', '.join((k))
    print '------------------------------------------'

cluster: 3159 abedinhstategov_sent, ap, newsmahogany, july, march
abedinhstategov_sent, ap, newsmahogany, july, march, h_hrodclintonemailcom, full_h, says, draft, full_abedin, cheryl_millscdstategov, go, next, annemarie, slaughter, state_case, release_part, day, news, say, going
------------------------------------------
cluster: 1154 draftdocx, haiti, cdm, june, january
draftdocx, haiti, cdm, june, january, october, april, november, update, nora, kennedy, daniel, patrick, iv, richard, thank, please, dec, information, talk, philip
------------------------------------------
cluster: 778 hillary, email, melanne, foreign, jan
hillary, email, melanne, foreign, jan, week, hope, verveer, think, thanks, much, message, ill, best, clinton, thx, let, judith, great, nov, women
------------------------------------------
cluster: 725 obama, policy, american, ut, world
obama, policy, american, ut, world, two, mr, administration, afghanistan, obamas, years, officials, year, global, even, former, mili

Ассесору предлагается прочитать вначале 5 слов, для понимания темы кластера. Затем читается текст длиной в 20 слов. Он раскрывает и уточняет тему. Затем ассессор может согласиться, что тема через эти слова выделяется достаточно конкретно, а затем решить, хорошо ли 5 слов ее описывают.

Также в последующих тестированиях ассессоров можно добавить одно из 5 слов другого кластера в 5 слов кластера и дать им найти лишнее слово среди пяти. Точность такого поиска и будет итоговым качество классификации. Это стоит делать, когда слова в кластерах получились достаточно чистыми. У меня такого не присутствует. Много шума.

В последующей работе можно убрать стоп-слова такие, как дни недели, названия месяцев, частицы "ок" и прочее. Убрать бессмысленный хлам типа "full_h". Для этого можно писать регулярные выражения и поставить длину слов со смыслом от 4 символов. Все это поможет сделать темы более осмысленными. 



Пока что нет идей, как сделать кластеры более равновесными. Возможно, такое может получиться при использовании особых методов классификации и параметров

Выделим все биграммы

In [37]:
cnt = CountVectorizer(ngram_range=(2,2))
new_text3 = cnt.fit_transform(new_text)

In [51]:
a = np.array(new_text3.sum(axis=0))[0]

In [65]:
voc2 = cnt.get_feature_names()

Упорядочим по частоте встречаемости и выведем топ по частоте

In [82]:
b=a.argsort()[::-1]

In [83]:
b[0]

98275

In [87]:
l=0
number = 0
while l<=9:
    i = b[number]
    number = number +1
    if '_' not in voc2[i]:
        print voc2[i]
        l=l+1

department state
us department
case doc
state case
doc date
date release
release part
mills cheryl
abedin huma
release full
